# 필요 모듈 및 라이브러리 설치

In [ ]:
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

!pip install sentencepiece
!pip install nltk

%tensorflow_version 1.x

import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf
import logging
from tensorflow.saved_model import tag_constants
import base64
import json

tf.__version__

Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 155320 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.27-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.27-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.27-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
     |████████████████████████████████| 1.2 MB 5.1 MB/s 
TensorFlow 1.x selected.


'1.15.2'

In [ ]:
# bert 모델 다운
# !git clone https://github.com/google-research/bert /content/Gdrive/bert_files/bert_model
# !git clone https://github.com/google-research/bert
sys.path.append('bert')

Cloning into 'bert'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 328.28 KiB | 3.42 MiB/s, done.
Resolving deltas: 100% (182/182), done.


In [ ]:
!cp ./bert /content/Gdrive/bert_files

cp: -r not specified; omitting directory './bert'


# Google Drive와 GCP 연동

In [ ]:
from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()

!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the 

In [ ]:
# Gdrive 폴더 만들기
!mkdir -p Gdrive

# google drvie와 colab 연동시키기
!google-drive-ocamlfuse Gdrive -o nonempty

# 파일 리스트 확인
!ls

adc.json  bert	Gdrive	sample_data


# TPU Address 확인

In [ ]:
# 현재 colab 런타임 환경이 TPU로 세팅돼 있는지 확인
assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'

# TPU 주소 추출
TPU_ADDRESS = 'grpc://'+os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

# 코랩의 인증 모듈로 할당된 TPU 모델 확인하기
from google.colab import auth
auth.authenticate_user()

with tf.Session(TPU_ADDRESS) as session:
    print('TPU devices:')
    pprint.pprint(session.list_devices())

    # TPU에 credentials 정보 저장하기
    with open('/content/adc.json', 'r') as f :
        auth_info = json.load(f)
    
    # google cloud storage(bucket)에 현재 TPU와 인증 정보 설정하기
    # 이 세팅을 해야 gcp bucket과 연동 가능
    tf.contrib.cloud.configure_gcs(session, credentials=auth_info)

TPU address is grpc://10.124.254.122:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 15763421417039151740),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 16172461483471682511),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 978365273433876448),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8957188161996833398),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 1165540587110668902),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 1018362212208980980),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 15075839950048221655),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 5507921427282504626),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 176511265341

# 환경변수 세팅

In [ ]:
# 학습할 모델 경로
MODEL_DIR = 'gs://nlp_pretrain_model/bert_model'

# 저장할 모델 경로(output)
PROD_MODEL_DIR='gs://nlp_pretrain_model/output'

# 단어사전 파일 경로
VOCAB_FILE= 'gs://nlp_pretrain_model/bert_model/vocab.txt'

# BERT 최대 길이
MAX_SEQ_LENGTH= 128

# 학습된 모델 경로
PRE_TRAINED_MODEL_PATH='gs://pretrained_files_1'

# 구글 드라이브 bert 모델 경로
GDRIVE_BERT_ROOT= 'Gdrive/bert_files'

# 구글드라이브 자연어 데이터 경로
GDRIVE_NLP_DATA_ROOT= 'Gdrive/bert_files'

# BERT 환경설정
BERT_CONFIG='gs://nlp_pretrain_model/bert_model/bert_config.json'

# 배치 크기
BATCH_SIZE=64

# 학습 데이터 가공 및 생성

In [ ]:
import pandas as pd
import gspread

gc = gspread.authorize(GoogleCredentials.get_application_default())

FILE_PATH = '/content/Gdrive/ONe/db/Q&A_개발활용가능_2020.10_sample.xlsx'

df = pd.read_excel(FILE_PATH)


In [ ]:
df.head()

,번호,종류\n(개/고양이),대분류,중분류,소분류,Question\n(질문 정리),Answer2 (답변 정리),Hashtag 2,Unnamed: 8,건강/의료,환경/생활,교육/심리,단순문의,반려상품,법률,입양/장례,Unnamed: 16,총합
0,1.0,개,건강/의료,기타,NaN,감기 걸린 강아지도 산책을 시켜야 하나요?,강아지에게 산책은 중요한 부분이지만 컨디션이 저조하거나 감기에 걸린 채로 나가는 건...,"#강아지, #감기, #산책, #놀이",NaN,319.0,265.0,317.0,73.0,21.0,3.0,70.0,NaN,1068.0
1,NaN,개,건강/의료,기타,NaN,암컷 강아지의 생리와 혈뇨는 어떻게 구분하나요?,생리도 혈뇨도 피가 나오기 때문에 혼동이 올 수 있어요. 하지만 두 증상에 확실한 ...,"#강아지, #생리, #혈뇨, #출혈, #차이점, #자궁축농증, #배뇨",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,개,건강/의료,기타,NaN,강아지도 생리를 하나요?,강아지도 사람과 마찬가지로 생리를 해요. 보통 생후 6개월~12개월 사이에 시작하고...,"#강아지, #초경, #생리, #생리주기, #생리기간, #생리대, #기저귀, #폐경",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,개,건강/의료,기타,NaN,강아지가 커피를 먹어도 되나요?,커피는 카페인 함량이 높아 동물들에게 치명적이에요. 섭취한 양을 확인하신 후 동물병...,"#강아지, #커피, #이물섭식, #카페인, #위험 #치명적 #섭취불가",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,개,건강/의료,기타,NaN,강아지는 방구를 참지 못하나요?,강아지는 방귀를 참기 어려울수 있어요.,#강아지방귀,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.tail()

,번호,종류\n(개/고양이),대분류,중분류,소분류,Question\n(질문 정리),Answer2 (답변 정리),Hashtag 2,Unnamed: 8,건강/의료,환경/생활,교육/심리,단순문의,반려상품,법률,입양/장례,Unnamed: 16,총합
1063,NaN,고양이,환경/생활,양육,NaN,고양이 발정기 증상이 뭐가 있나요?,고양이 발정기 증상은\n남자고양이는 특히 스프레이 라는 행동을 해요. 주변을 돌아다...,#암컷고양이 #수컷고양이 #남자고양이 #여자고양이 #고양이발정기증상 #발정기 #발정...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1064,NaN,고양이,환경/생활,양육,NaN,고양이가 계속 우는데 발정기가 온건가요?,고양이 발정기 증상은\n남자고양이는 특히 스프레이 라는 행동을 해요. 주변을 돌아다...,#암컷고양이 #수컷고양이 #남자고양이 #여자고양이 #고양이발정기증상 #발정기 #발정...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1065,NaN,고양이,환경/생활,양육,NaN,고양이 비만인가요? 비만인데 어떻게 관리하면 될까요?,비만도를 체크하려면 고양이를 위에서 봤을 때 타원형 체형이면 비만이라고 보면 돼요....,#비만고양이 #뚱냥이 #고양이위에서타원형비만 #뼈만져지면정상 #식단관리 #식단조절 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1066,NaN,고양이,환경/생활,양육,NaN,고양이 비만인것 같은데 어떻게 관리하면 될까요?,고양이 건강을 위해서 자율 급식보다는 제한 급식으로 식단을 조절해야 해요.\n\n고...,#비만고양이 #뚱냥이 #식단관리 #식단조절 #제한급식 #캣휠 #고양이장난감 #고양이...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1067,NaN,고양이,환경/생활,환경/생활,기타,고양이를 키울 때 주의해야 하는 건 어떤 게 있나요?,"① 빗질, 발톱 다듬기 등 건강과 미용을 위한 피모관리\n고양이는 강아지와 다르게 ...","#고양이, #기본관리, #주의사항",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 컬럼명 변경 (종류)
re_columns = ['번호', '종류', '대분류', '중분류', '소분류', 'Question',
       'Answer', 'Hashtag 2', 'Unnamed: 8', '건강/의료', '환경/생활', '교육/심리',
       '단순문의', '반려상품', '법률', '입양/장례', 'Unnamed: 16', '총합']
df.columns = re_columns

df.head()

,번호,종류,대분류,중분류,소분류,Question,Answer,Hashtag 2,Unnamed: 8,건강/의료,환경/생활,교육/심리,단순문의,반려상품,법률,입양/장례,Unnamed: 16,총합
0,1.0,개,건강/의료,기타,NaN,감기 걸린 강아지도 산책을 시켜야 하나요?,강아지에게 산책은 중요한 부분이지만 컨디션이 저조하거나 감기에 걸린 채로 나가는 건...,"#강아지, #감기, #산책, #놀이",NaN,319.0,265.0,317.0,73.0,21.0,3.0,70.0,NaN,1068.0
1,NaN,개,건강/의료,기타,NaN,암컷 강아지의 생리와 혈뇨는 어떻게 구분하나요?,생리도 혈뇨도 피가 나오기 때문에 혼동이 올 수 있어요. 하지만 두 증상에 확실한 ...,"#강아지, #생리, #혈뇨, #출혈, #차이점, #자궁축농증, #배뇨",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,개,건강/의료,기타,NaN,강아지도 생리를 하나요?,강아지도 사람과 마찬가지로 생리를 해요. 보통 생후 6개월~12개월 사이에 시작하고...,"#강아지, #초경, #생리, #생리주기, #생리기간, #생리대, #기저귀, #폐경",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,개,건강/의료,기타,NaN,강아지가 커피를 먹어도 되나요?,커피는 카페인 함량이 높아 동물들에게 치명적이에요. 섭취한 양을 확인하신 후 동물병...,"#강아지, #커피, #이물섭식, #카페인, #위험 #치명적 #섭취불가",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,개,건강/의료,기타,NaN,강아지는 방구를 참지 못하나요?,강아지는 방귀를 참기 어려울수 있어요.,#강아지방귀,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- 카테고리 별로 분류 파싱하여 카테고리 트리 만들기

In [ ]:
category_dict = {}
category_list = []

category_list.append(0)
idx=1

for i, row in df.iterrows() :

    if row['종류'] != '개' and row['종류'] != '고양이':
        continue
    if row['종류'] not in category_dict :
        category_dict[row['종류']] = {}
    if row['대분류'] not in category_dict[row['종류']]:
        category_dict[row['종류']][row['대분류']] = {}
    if row['중분류'] not in category_dict[row['종류']][row['대분류']]:
        category_dict[row['종류']][row['대분류']][row['중분류']] = {}
    if row['소분류'] not in category_dict[row['종류']][row['대분류']][row['중분류']]:
        category_dict[row['종류']][row['대분류']][row['중분류']][row['소분류']] = idx

        category_list.append(str(row['종류']) + ':' + str(row['대분류']) + ':' + str(row['중분류']) + ':' + str(row['소분류']))
        idx += 1

print(category_dict)
print(category_list)
print(len(category_list))

{'개': {'건강/의료': {'기타': {nan: 1}, '치료': {'간단건강체크': 2, '구조/구호': 3, '수술': 4, '예방/약': 5, '예방접종': 6, '응급처치': 7, '중성화': 8, '질병/증상': 9, '질병/치료': 10}}, '교육/심리': {'감정/심리': {nan: 11}, '놀이/산책': {nan: 12}, '행동': {nan: 13}, '훈련/행동교정': {nan: 14}}, '단순문의': {'기타': {nan: 15}, '보호자 고민': {nan: 16}, '품종문의': {nan: 17}}, '반려상품': {'기본용품': {'목욕/미용용품': 18}, '식품': {'건강관리': 19, '사료/간식': 20}}, '법률': {'사람/동물': {nan: 21}, '의료사고': {nan: 22}}, '입양/장례': {'기타': {nan: 23}, '입양/분양': {'기타': 24, '입양 받을 때': 25, '입양 보낼 때': 26, '입양받을때': 27}, '장례': {'장례': 28}}, '환경/생활': {'교배/출산': {'교배': 29, '임신/출산': 30}, '급여/간식': {'급여방법': 31, '기타': 32}, '기타': {nan: 33}, '놀이/산책': {nan: 34}, '양육': {'기타': 35, '목욕/미용': 36, '양육': 37, nan: 38}, '환경/생활': {'기타': 39}}}, '고양이': {'건강/의료': {'기타': {nan: 40}, '치료': {'간단건강체크': 41, '구조/구호': 42, '수술': 43, '예방/약': 44, '예방접종': 45, '응급': 46, '응급처치': 47, '중성화': 48, '질병/증상': 49, '질병/치료': 50}}, '교육/심리': {'감정/심리': {nan: 51}, '기타': {nan: 52}, '놀이/산책': {nan: 53}, '행동': {nan: 54}, '훈련/행동교정': {nan: 55}}, '단순문의': {'기타': {

In [ ]:
NUM_LABELS = len(category_list)

### 데이터 전처리

In [ ]:
import re

PATTERN = '[^\w\s]'
PATTERN_MULTI_SPACE = ' +'
PATTERN_LINE_BREAKER = '\n|\r'
REPL = " "

def isNaN(string):
    return string != string

train_raw = {
    'label': [],
    'alpha': [],
    'text': []
}

continued_value = {
    '종류' : [],
    '대분류' : [],
    '중분류' : [],
    '소분류' : [],
    'Question' : []
}

# Question
for i, row in df.iterrows():
    if isNaN(row['Question']) : row['Question'] = REPL
    if row['종류'] != '개' and row['종류'] != '고양이' and row['Question'] != '' and row['Question'] != 'None':
        continue
    else:
        try:
            qu = re.sub(pattern=PATTERN, repl=REPL, string=str(row['Question']))
            qu = re.sub(pattern=PATTERN_LINE_BREAKER, repl=REPL, string=qu)
            qu = re.sub(pattern=PATTERN_MULTI_SPACE, repl=REPL, string=qu)

            if(qu != REPL) : qu = qu.strip()
            train_raw['label'].append(category_dict[row['종류']][row['대분류']][row['중분류']][row['소분류']])
            train_raw['alpha'].append('a')
            train_raw['text'].append(qu)
        
        except:
            continued_value['종류'].append(row['종류'])
            continued_value['대분류'].append(row['대분류'])
            continued_value['중분류'].append(row['중분류'])
            continued_value['소분류'].append(row['소분류'])
            continued_value['Question'].append(row['Question'])

# Answer
for i, row in df.iterrows():
    if isNaN(row['Answer']): 
        row['Answer'] = REPL
    if row['종류'] != '개' and row['종류'] != '고양이' and row['Answer'] != '' and row['Answer'] != 'None':
        continue
    else :
        try:
            qu = re.sub(pattern=PATTERN, repl=REPL, string=str(row['Answer']))
            qu = re.sub(pattern=PATTERN_LINE_BREAKER, repl=REPL, string=qu)
            qu = re.sub(pattern=PATTERN_MULTI_SPACE, repl=REPL, string=qu)
            if(qu != REPL) : qu = qu.strip()
            qu = qu[:256] if len(qu) > 256 else qu 
            train_raw['label'].append(category_dict[row['종류']][row['대분류']][row['중분류']][row['소분류']])
            train_raw['alpha'].append('a')
            train_raw['text'].append(qu)
        except:
            continued_value['종류'].append(row['종류'])
            continued_value['대분류'].append(row['대분류'])
            continued_value['중분류'].append(row['중분류'])
            continued_value['소분류'].append(row['소분류'])
            continued_value['Question'].append(row['Question'])

train =pd.DataFrame(train_raw)

train = train.reset_index() # ==> index 재설정
train = train.rename(columns={'index':'id'}) # index라는 column을 id로 변동

# 에러 난 값들
error_values = pd.DataFrame(continued_value)

In [ ]:
train.head(2)

,id,label,alpha,text
0,0,1,a,감기 걸린 강아지도 산책을 시켜야 하나요
1,1,1,a,암컷 강아지의 생리와 혈뇨는 어떻게 구분하나요


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2136 entries, 0 to 2135
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2136 non-null   int64 
 1   label   2136 non-null   int64 
 2   alpha   2136 non-null   object
 3   text    2136 non-null   object
dtypes: int64(2), object(2)
memory usage: 66.9+ KB


### 데이터 분리

In [ ]:
from sklearn.model_selection import train_test_split
from pandas import DataFrame
import numpy as np

df_bert_train, df_bert_dev = train_test_split(train,test_size=0.1)

test = df_bert_dev[['id', 'text']]

test.head()
test.info()

df_bert_test =test
df_bert_test

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214 entries, 126 to 969
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      214 non-null    int64 
 1   text    214 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.0+ KB


,id,text
126,126,털을 밀고 기르는 중인데 일부분이 빳빳하고 누렇게 변색되었는데 왜 그럴까요
1755,1755,고양이 턱드름을 손으로 짜거나 칫솔로 문지르게 되면 2차 감염으로 염증을 유발 할 ...
594,594,강아지를 비행기에 태우려면 동물등록을 꼭 해야 하나요
43,43,새끼강아지 인데 예방접종 1차까지 완료했는데 산책 언제 해도 될까요
1763,1763,자주 다니는 길 이라면 퇴근할 때 마다 잠깐씩 들러서 찾아보세요 고양이가 다쳤다면 ...
...,...,...
1232,1232,장이 아파서 그럴 수 있어요 구토 원인으로 급성위염 기생충감염 췌장염 장폐색 장중첩...
629,629,새끼고양이 심장에서 그르렁 진동소리가 느껴지는데 정상인가요
1858,1858,고양이가 골골소리를 낼 경우는기분이 좋거나 편안할 때 내는 소리예요 일부에서 불안정...
708,708,고양이 중성화수술 몇개월 때 하면 될까요


# Generate train, dev & test file
- test file header=True (컬럼명을 살린다는 의미)
- other files header=false (컬럼명을 없앤다는 의미)

In [ ]:
df_bert_train.to_csv(GDRIVE_BERT_ROOT + '/data/train.tsv', sep='\t', index=False, header=False, encoding='UTF-8')
df_bert_dev.to_csv(GDRIVE_BERT_ROOT + '/data/dev.tsv', sep='\t', index=False, header=False, encoding='UTF-8')
df_bert_test.to_csv(GDRIVE_BERT_ROOT + '/data/test.tsv', sep='\t', index=False, header=True, encoding='UTF-8')
error_values.to_csv(GDRIVE_BERT_ROOT + '/data/error_values.tsv', sep='\t', index=False, header=True, encoding='UTF-8')

with open('{}/category.json'.format(GDRIVE_BERT_ROOT + '/data'), 'w') as fo:
    json.dump(category_list, fo, indent=2)

# Classify 학습 트레이닝
- Fine-tuning for Classifier
- Added checkpoint

In [ ]:
# 환경변수 세팅 (그런데 왜 하는걸까?)
os.environ['GDRIVE_BERT_ROOT'] = GDRIVE_BERT_ROOT
os.environ['MAX_SEQ_LENGTH'] = str(MAX_SEQ_LENGTH)
os.environ['VOCAB_FILE'] = VOCAB_FILE
os.environ['BERT_CONFIG'] = BERT_CONFIG
os.environ['MODEL_DIR'] = MODEL_DIR
os.environ['PROD_MODEL_DIR'] = PROD_MODEL_DIR
os.environ['NUM_LABELS'] = str(NUM_LABELS)

In [ ]:
!python $GDRIVE_BERT_ROOT/bert_model/run_classifier.py \
--task_name=cola \
--do_train=True \
--do_predict=False \
--do_eval= True \
--data_dir = $GDRIVE_BERT_ROOT/data \
--vocab_file=$VOCAB_FILE \
--bert_config_file=$BERT_CONFIG \
--output_dir=$MODEL_DIR \
--max_seq_length=$MAX_SEQ_LENGTH \
--train_batch_size=$BATCH_SIZE \
--learning_rate=2e-5 \
--num_train_epochs=3 \
--num_labels=$NUM_LABELS \
--use_tpu=True \
--tpu_address=$TPU_ADDRESS \
--do_lower_case=False